In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
df = pd.read_excel(r"C:\Users\Admin\data stb\mock survey data 3.1.xlsx")

In [6]:
df

,case,year,month,Country of Residence,City of Residence,Purpose of Visit,Main Purpose of Visit,Weights_QTR,Air_Terminal,Sea_Terminal,...,Travel companion - Alone,Travel companion - Spouse,Travel companion - Your Child/Children,Travel companion - Parents/Parents-in-law,Travel companion - Grandparents/Grandparents-in-law,Travel companion - Siblings,Travel companion - Other relatives,Travel companion - Friends,Travel companion - Business associates/Colleagues,Travel companion - Others
0,1.0,2018,1,India,Delhi,Leisure,Holiday/ Rest & Relax,660.046512,2.0,NaN,...,0,1,0,0,0,0,0,0,0,0
1,2.0,2018,1,India,Delhi,Leisure,Visiting friends/ relatives (who are not inter...,433.217949,2.0,NaN,...,1,0,0,0,0,0,0,0,0,0
2,3.0,2018,1,United Arab Emirates,Dubai,Leisure,General business purpose,335.125000,2.0,NaN,...,1,0,0,0,0,0,0,0,0,0
3,4.0,2018,1,India,Others,Leisure,Holiday/ Rest & Relax,490.977273,2.0,NaN,...,0,1,1,0,0,0,0,0,0,0
4,5.0,2018,1,Canada,Toronto,Business + Accompanying Pax,Visiting an international student (relative or...,379.800000,2.0,NaN,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22969,7789.0,2018,12,Australia,Brisbane,Leisure,Holiday/ Rest & Relax,310.354839,2.0,NaN,...,0,1,0,0,0,0,0,0,0,0
22970,7790.0,2018,12,Hong Kong,Hong Kong,Leisure,Holiday/ Rest & Relax,109.393939,2.0,NaN,...,0,0,0,0,0,0,0,1,0,0
22971,7791.0,2018,12,Hong Kong,Hong Kong,Leisure,Holiday/ Rest & Relax,326.906250,2.0,NaN,...,0,1,0,0,0,0,0,0,0,0
22972,7792.0,2018,12,Hong Kong,Hong Kong,Business + Accompanying Pax,Gather information/ facts on the education ser...,820.615385,2.0,NaN,...,1,0,0,0,0,0,0,0,0,0


In [28]:
# Overview
print("Data Shape:", df.shape)
num_columns = len(df.columns)
print("Total number of columns:", num_columns)

print("\n")
print("Data Columns:", df.columns)



print("\n")
print("\nColumn Information:")
df.info()

Data Shape: (22974, 51)
Total number of columns: 51


Data Columns: Index(['case', 'year', 'month', 'Country of Residence', 'City of Residence',
       'Purpose of Visit', 'Main Purpose of Visit', 'Weights_QTR',
       'Air_Terminal', 'Sea_Terminal', 'Land_Terminal',
       'First Visit To Singapore', 'Length of Stay', 'Travel Type', 'Gender',
       'Marital Status', 'Occupation', 'Occupation.1', 'Other Designation',
       'Designation (free text)', 'shopping_fash', 'shopping_jewllery',
       'shopping_watches', 'shopping_wellness', 'shopping_food',
       'shopping_gifts', 'shopping_consumertech', 'shopping_antiques',
       'shopping_others', 'shopping_any', 'totacc', 'totfnb', 'tottran',
       'totbiz', 'totedu', 'totmedi', 'tototh', 'totshopping_rep', 'tot.exp',
       'c4a_1', 'MainHotel', 'Travel companion - Alone',
       'Travel companion - Spouse', 'Travel companion - Your Child/Children',
       'Travel companion - Parents/Parents-in-law',
       'Travel companion - Grand

In [15]:
# Define expected number of rows
expected_rows = 22974

# Initialize lists to store column names
full_columns = []
incomplete_columns = []

# Loop through each column
for col in df.columns:
    non_null_count = df[col].notnull().sum()
    if non_null_count == expected_rows:
        full_columns.append(col)
    else:
        incomplete_columns.append((col, non_null_count))

# Print columns with all values
print("**Columns with FULL data (22,974 rows):**")
for col in full_columns:
    print(f"- {col}")

print("\n")

print("**Columns with MISSING data:**")
for col, count in incomplete_columns:
    print(f"- {col}: {count} non-null values")


**Columns with FULL data (22,974 rows):**
- year
- month
- Country of Residence
- Main Purpose of Visit
- Weights_QTR
- First Visit To Singapore
- Length of Stay
- Travel Type
- Gender
- Marital Status
- Occupation
- shopping_jewllery
- shopping_watches
- shopping_wellness
- shopping_consumertech
- shopping_antiques
- totacc
- totfnb
- tottran
- totbiz
- totedu
- totmedi
- tototh
- totshopping_rep
- tot.exp
- c4a_1
- Travel companion - Alone
- Travel companion - Spouse
- Travel companion - Your Child/Children
- Travel companion - Parents/Parents-in-law
- Travel companion - Grandparents/Grandparents-in-law
- Travel companion - Siblings
- Travel companion - Other relatives
- Travel companion - Friends
- Travel companion - Business associates/Colleagues
- Travel companion - Others


**Columns with MISSING data:**
- case: 22972 non-null values
- City of Residence: 22962 non-null values
- Purpose of Visit: 20641 non-null values
- Air_Terminal: 17977 non-null values
- Sea_Terminal: 2381 non-

In [30]:
print("\nMissing Values per Column:")
print(df.isnull().sum())


Missing Values per Column:
case                                                       2
year                                                       0
month                                                      0
Country of Residence                                       0
City of Residence                                         12
Purpose of Visit                                        2333
Main Purpose of Visit                                      0
Weights_QTR                                                0
Air_Terminal                                            4997
Sea_Terminal                                           20593
Land_Terminal                                          20359
First Visit To Singapore                                   0
Length of Stay                                             0
Travel Type                                                0
Gender                                                     0
Marital Status                                           

## Data dictionary

- case - **Unique respondent ID**
- year - **Date of survey response (year)**
- month -	**Date of survey response (year and month)**
- Country of Residence- **Visitor’s country of origin**
- City of Residence	- **Visitor’s home city (if provided)**
- Main Purpose of Visit -	**Primary reason for their trip (e.g. Leisure, Business, MICE)**
- Weights_QTR - **Survey weight to project sample to population**
- Air_Terminal - **Mode of travel - Airplane**
- Sea_Terminal - **Mode of travel - Sea**
- Land_Terminal - **Mode of travel - Land**
- First Visit To Singapore - **Whether this is their first trip to Singapore (Yes/No)**
- Length of Stay - **Number of nights stayed**
- Travel Type - **Non package tour vs. packaged tour**
- Gender
- Marital Status
- Occupation 
- shopping_fash - **Spend on fashion items**
- shopping_jewllery - **Spend on jewllery items**
- shopping_watches - **Spend on watches items**
- shopping_wellness - **Spend on wellnes items**
- shopping_food - **Spend on food**
- shopping_gifts - **Spend on gifts**
- shopping_consumertech - **Spend on tech**
- shopping_antiques - **Spend on antiques**
- shopping_others - **shopping others**
- shopping_any - **shopping any**
- totacc - **Total spend on accommodation**
- totfnb - **Total spend on food & beverage**                                        
- tottran - **Total spend on transport**                                                           
- totbiz - **Total spend on Total Business Spend**                                                     
- totedu - **Total spend on Total Education Spend**                                                  
- totmedi - **Total spend on Total Medical Spend**                    
- tototh - **Total spend on Total Other Spend**                 
- totshopping_rep - **Total spend on Total Reported Shopping Spend (Most likely uses the shopping variables e.g. shopping_fash etc to calucate this)**
- tot.exp - **Total spend on 	Total Expenditure across all categories**
- c4a_1 **(Not sure yet)**
- MainHotel	- **Categorical code for hotel class or brand**
- Travel companion – **Binary for each companion type (alone, spouse, Your child/children, Parents/Parents-in-law, Grandparents/Grandparents-in-law,
  Siblings, Other relatives, Friends, Business associates/Colleagues, Others.)**

Comments and note to self
1. Total spend on Total Reported Shopping Spend (Most likely uses the shopping variables e.g. shopping_fash etc to calucate this)
2. Travel companion - others most likely gf/bf etc - can be not engaged.
3. c4a_1 - need to do exploratory for this.
4. clustering model most likely do not need to look at shopping, just focus on total expenses
5. ideas for feature - tot.exp / Length of Stay (understand daily expenses), age band (1-19, 20-40 etc), length of stay (< 3 days, 4 to 7 days, 7 days to 14 days, > 14 days)
6. change date and time

## Problem statement

Candidate Name:
Please note the following:

1)	Both questions use the provided datasets
  
2)	All source code must be provided with comments along with instructions on how to execute the code.
  
3)	R is preferred, but you are free to consider other tools. Regardless of the tool you use, source code / workings must be provided and clearly documented

  
4)	Refer to mock survey data 3.1.xls for Question 1 and 2.

  
5)	You are free to augment this dataset with publicly available data to sharpen your insights / recommendations.  However, if you choose to do so, you must document the reasons and how you are using the data to augment your insights / recommendations.

  
6)	You can refer to the sheet “dictionary” for a description of the fields to aid in your analysis.

Background
The marketing team is looking to refresh their marketing strategy and reached out to the Data Science team for help.  A survey was commissioned to collect visitor data over a period of 12 months to understand their spending behaviours and preferences.  You are now in-charge of analysing this data and presenting visitor behaviour insights to help the inform the marketing team on their strategy to increase visitorship and spend in Singapore.

Question 1

**With reference to the mock survey dataset (mock survey data 3.xlsx),** 

a.	Describe the analytical approach you will take and data fields you would look into when it comes to doing exploratory data analysis.

b.	Highlight the data idiosyncrasies / issues you found in this dataset and how would you address the issues identified.

c.	What are the considerations that you will take when analysing survey data.

Use this data set to illustrate and explain your approach and considerations.  Please include your supporting scripts/output from R or an alternative tool of your choice. 

Question 2

Using the mock survey data (mock survey data 3.1.xlsx), answer the following questions.

1)	**You are given a set of survey data which captures spend amounts among other data points.  The Marketing team is curious to understand the different segments of visitors who are coming to Singapore. You are given the task to analyse the survey data so that the marketing team can formulate data driven strategies to draw in more visitors to Singapore.**

Using the set of data provided, answer ALL of the following questions:

a.	How many different segment of visitors can be identified from the survey data

b.	Please explain the choice of metric you used

c.	What are the key assumptions you have taken to identify the different segments?

d.	What were the other approaches you considered? Please explain the reason for the technique / approach used as well as the pros and cons.
e.	Why is your analytical approach performing well / not well?

f.	Was any feature engineering required? If yes, what were they. If no, why?

 2)	**Based on the analysis done in (1), answer the following business questions:**
  
a.	Describe and profile the segment tourists that you have identified

b.	What can we learn about our visitors from the survey data as well as the visitor segments that will help the Marketing team better market Singapore as a tourist destination them?

c.	Is there a strong or weak relationship between travel companions and choice of hotel?  Please provide an explanation on how you arrived at your conclusion.

3)	**With your findings from (Questions 1, 2.1 and 2.2), prepare a concise PowerPoint presentation to pitch your strategy to the Marketing Team on how will your insights help STB attract visitors to spend in Singapore. Hints:**
•	You are encouraged to explore the data beyond the questions asked in (a) – (c)

•	You should use your analysis results from question 1 to support your recommendations

•	Use appropriate charts / visuals to communicate your story

•	Be careful with the chart-ink ratio


In [31]:
# Filter rows where all three terminal fields are missing
no_terminal_df = df[
    df['Air_Terminal'].isnull() &
    df['Sea_Terminal'].isnull() &
    df['Land_Terminal'].isnull()
]

# Show the result
print(f"Number of rows with NO terminal data: {len(no_terminal_df)}")
display(no_terminal_df)


Number of rows with NO terminal data: 3


,case,year,month,Country of Residence,City of Residence,Purpose of Visit,Main Purpose of Visit,Weights_QTR,Air_Terminal,Sea_Terminal,...,Travel companion - Alone,Travel companion - Spouse,Travel companion - Your Child/Children,Travel companion - Parents/Parents-in-law,Travel companion - Grandparents/Grandparents-in-law,Travel companion - Siblings,Travel companion - Other relatives,Travel companion - Friends,Travel companion - Business associates/Colleagues,Travel companion - Others
695,699.0,2018,1,Vietnam,Hanoi,Leisure,Holiday/ Rest & Relax,1001.000000,NaN,NaN,...,0,1,0,0,0,0,0,0,0,1
806,811.0,2018,1,Thailand,Chiang Mai,Leisure,Visiting friends/ relatives (who are not inter...,625.809524,NaN,NaN,...,0,0,0,1,0,0,0,0,0,0
819,824.0,2018,1,Italy,Others,Leisure,Holiday/ Rest & Relax,514.500000,NaN,NaN,...,0,1,1,0,0,0,0,0,0,0
